# **Time-Series Hierarchial Clustering**

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Each observation in the dataset has a target variable and 6 time series, each of which is 15 time steps long.  The target variable has 6 classes.

## Have a peak at the dataset. A time-series is represented by 15 columns: X1,X2, X3 ...... X15; where X is A,B,C,D,E or F

In [32]:
file = open('lp5.data', 'r')
lines = file.readlines()



df_cols = []

for i in range(6):
  prefix = chr(65 + i)

  for j in range(15):
   df_cols.append(prefix + str(j+1))      

df_cols.append("Target")
target = None
df_row = []
df_rows = []
l = [[],[],[],[],[],[]]

for line in lines:
  tokens = line.split()
  if(len(tokens) == 0):
    if l[0]:
      for i in range(6):
        df_row.extend(l[i])  
        
      df_row.append(target)  
      df_rows.append(df_row)
      df_row = []
      l = [[],[],[],[],[],[]]
    continue

  if(len(tokens) == 1):
    target = tokens[0]

  else:
    for i in range(6):
      l[i].append(int(tokens[i]))

df = pd.DataFrame(data = df_rows, columns = df_cols)
df_ = df.copy()
df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,B15,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,D7,D8,D9,D10,D11,D12,D13,D14,D15,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,E11,E12,E13,E14,E15,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,Target
0,-2,-2,-2,-2,-3,-2,-2,-2,-2,-2,-2,-1,-2,-3,-2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,81,79,79,80,79,81,80,79,79,79,...,0,0,0,0,1,0,0,1,0,-5,-4,-4,-4,-5,-5,-4,-4,-4,-4,-4,-4,-5,-4,-4,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,normal
1,6,42,-5,-4,2,0,-3,-6,-3,-2,-4,2,-1,-4,-4,-1,-3,4,-17,1,-5,-7,2,3,-9,2,-3,-5,5,1,79,80,74,93,74,78,83,71,76,86,...,8,-12,-6,16,-5,7,6,-14,-3,4,53,-10,-6,-3,-6,-5,-11,-9,-8,-7,1,-6,-9,-12,-3,3,-1,-3,-4,0,-3,1,-4,-2,2,9,0,-4,5,normal
2,-2,0,-4,-2,-4,-2,1,-8,-1,-3,-3,-6,0,-5,-6,-6,2,-5,-2,4,4,-9,3,0,-6,8,-1,-9,5,-10,85,74,76,78,76,74,87,69,83,83,...,13,-13,-2,6,-19,-1,13,-17,16,-5,1,-11,-6,-13,-9,-2,-20,-6,-14,-11,-20,-5,-16,-14,2,0,4,1,-10,-4,-10,-5,-8,-5,-3,6,2,7,-1,normal
3,-2,-5,0,-2,-6,2,-1,1,-3,-1,-5,-4,1,-9,-1,-10,4,1,-6,3,-9,-8,3,-8,-1,4,-14,-7,6,-9,86,73,79,87,73,84,82,78,88,81,...,6,-11,5,-4,-17,17,3,-18,11,-7,-18,-10,-12,-21,-6,-8,-2,-13,-10,-21,-17,-7,-28,-15,-5,-2,-4,-1,-5,-9,0,-5,-3,-13,-15,-4,-2,-9,-11,normal
4,0,-2,2,1,1,-5,-1,-6,-5,-1,-3,-4,-2,3,-7,2,4,-9,-3,-1,-6,-6,-2,-10,5,5,-11,5,-3,-13,74,74,85,77,71,83,77,77,90,75,...,-2,-10,7,-23,-19,11,-18,-1,12,-9,-15,-4,-11,-2,-21,-17,-25,-21,-16,-19,-14,-11,-7,-30,-5,-3,-6,-10,-3,-5,-11,-10,-9,-6,-6,-4,-9,-1,-3,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,-5,-4,-4,-4,-4,-2,-4,-2,-1,-1,-1,-4,-2,-1,-1,14,1,0,-2,0,-1,0,-1,1,1,1,0,-1,1,1,-43,6,12,3,10,-2,10,-2,12,2,...,-2,-1,-3,-2,-3,-2,-1,-3,-2,-49,-2,-1,-1,-2,1,-2,1,1,2,1,-2,1,1,2,17,-4,-4,-3,-4,-3,-4,-3,-3,-3,-3,-4,-3,-3,-3,collision_in_tool
160,36,25,22,17,7,4,-5,1,1,3,0,3,1,1,2,-4,2,-14,-23,-8,6,11,-1,0,0,2,-1,-1,-1,-1,19,25,13,10,7,8,15,4,4,-3,...,-12,3,0,1,-3,3,1,1,1,53,37,35,27,13,9,0,5,5,7,2,5,4,4,5,-2,-3,-2,-2,-3,-3,-4,-3,-3,-3,-3,-4,-3,-3,-4,collision_in_tool
161,-12,-12,-8,-4,-6,-7,-5,1,2,-1,-1,2,0,1,1,17,12,3,3,6,3,-3,0,1,2,2,-1,1,2,2,3,11,6,1,9,3,5,-1,6,10,...,10,4,2,1,1,5,1,0,0,-10,-16,-11,-6,-5,-10,-7,2,4,1,1,3,1,3,3,-4,-4,-4,-3,-5,-3,-3,-3,-4,-4,-4,-3,-3,-4,-4,collision_in_tool
162,-41,-32,-21,-3,0,6,15,-1,-1,-2,-1,-3,-4,-4,-2,21,21,12,-1,-1,-2,-5,1,1,2,1,4,4,3,3,-5,-6,-6,2,13,14,3,5,6,5,...,15,3,4,1,1,-2,0,2,0,-59,-45,-31,-4,1,9,22,1,0,-2,0,0,-1,-3,-2,-4,-4,-4,-4,-3,-4,-3,-4,-3,-3,-3,-3,-3,-3,-3,collision_in_tool


In [0]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

def dtw(x,y):
  distance = 0
  for i in range(6):
    distance += fastdtw(x[15*i:15*i + 15], y[15*i:15*i + 15], dist=euclidean)[0]
  return distance

In [0]:
def get_dist_matrix(fun):
  org_dists = []

  for i in range(len(df_)):
    for j in range(len(df_)):
      if j > i:
        org_dists.append(fun(df_.iloc[i][:-1], df_.iloc[j][:-1]))

  return org_dists      

In [0]:
import fastcluster as fc
from scipy.cluster.hierarchy import cophenet

def get_linkage(fun): 
  org_dists = get_dist_matrix(fun)

  Z = fc.linkage(org_dists, metric = fun )         
  cophe_corr, _ = cophenet(Z, org_dists)

  return Z, cophe_corr, org_dists


In [0]:
from scipy.cluster.hierarchy import fcluster

Z_dtw, cophe_dtw, dist_dtw = get_linkage(dtw)
Z_euc, cophe_euc, dist_euc = get_linkage(euclidean)

## Single linkage clustering. The results are at the last 2 columns of the table below. Best results were obtained at inconsistency coefficient value 1.15

In [37]:
df['Cluster_DTW'] = fcluster(Z_dtw, 1.15)
df['Cluster_Euclidean'] = fcluster(Z_euc, 1.15)
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,B15,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,D9,D10,D11,D12,D13,D14,D15,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,E11,E12,E13,E14,E15,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,Target,Cluster_DTW,Cluster_Euclidean
0,-2,-2,-2,-2,-3,-2,-2,-2,-2,-2,-2,-1,-2,-3,-2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,81,79,79,80,79,81,80,79,79,79,...,0,0,1,0,0,1,0,-5,-4,-4,-4,-5,-5,-4,-4,-4,-4,-4,-4,-5,-4,-4,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,normal,30,10
1,6,42,-5,-4,2,0,-3,-6,-3,-2,-4,2,-1,-4,-4,-1,-3,4,-17,1,-5,-7,2,3,-9,2,-3,-5,5,1,79,80,74,93,74,78,83,71,76,86,...,-6,16,-5,7,6,-14,-3,4,53,-10,-6,-3,-6,-5,-11,-9,-8,-7,1,-6,-9,-12,-3,3,-1,-3,-4,0,-3,1,-4,-2,2,9,0,-4,5,normal,30,10
2,-2,0,-4,-2,-4,-2,1,-8,-1,-3,-3,-6,0,-5,-6,-6,2,-5,-2,4,4,-9,3,0,-6,8,-1,-9,5,-10,85,74,76,78,76,74,87,69,83,83,...,-2,6,-19,-1,13,-17,16,-5,1,-11,-6,-13,-9,-2,-20,-6,-14,-11,-20,-5,-16,-14,2,0,4,1,-10,-4,-10,-5,-8,-5,-3,6,2,7,-1,normal,30,10
3,-2,-5,0,-2,-6,2,-1,1,-3,-1,-5,-4,1,-9,-1,-10,4,1,-6,3,-9,-8,3,-8,-1,4,-14,-7,6,-9,86,73,79,87,73,84,82,78,88,81,...,5,-4,-17,17,3,-18,11,-7,-18,-10,-12,-21,-6,-8,-2,-13,-10,-21,-17,-7,-28,-15,-5,-2,-4,-1,-5,-9,0,-5,-3,-13,-15,-4,-2,-9,-11,normal,30,10
4,0,-2,2,1,1,-5,-1,-6,-5,-1,-3,-4,-2,3,-7,2,4,-9,-3,-1,-6,-6,-2,-10,5,5,-11,5,-3,-13,74,74,85,77,71,83,77,77,90,75,...,7,-23,-19,11,-18,-1,12,-9,-15,-4,-11,-2,-21,-17,-25,-21,-16,-19,-14,-11,-7,-30,-5,-3,-6,-10,-3,-5,-11,-10,-9,-6,-6,-4,-9,-1,-3,normal,30,10


## Evaluation results for both Dynamic Time Warping(DTW) and Euclidean Distance are calculated.

In [38]:
def print_results():

  print("No. of clusters formed for DTW: ", df['Cluster_DTW'].max())
  print("No. of clusters formed for Euclidean: ", df['Cluster_Euclidean'].max() )
  print()

  print("Cophenet Correlation of DTW: " ,cophe_dtw)
  print("Cophenet Correlation of Euclidean: " ,cophe_euc)  
  print()

  from sklearn import metrics
  from scipy.spatial.distance import squareform

  cluster_cols = ['Cluster_DTW', 'Cluster_Euclidean']

  for i in cluster_cols:
    print("*************" + i + "***************")
    print()
    print("Adjusted Rand Index: " ,metrics.adjusted_rand_score(df["Target"], df[i]))
    print("Adjusted Mutual Info Score: ", metrics.adjusted_mutual_info_score(df["Target"], df[i]))
    print("Normalized Mutual Info Score: ", metrics.normalized_mutual_info_score(df["Target"], df[i]))  
    print("Homogeneity, Completeness, V-measure: ", metrics.homogeneity_completeness_v_measure(df["Target"], df[i]))
    print("Fowlkes Mallows Score: ", metrics.fowlkes_mallows_score(df["Target"], df[i]))
    print()
    print()

print_results()  

No. of clusters formed for DTW:  66
No. of clusters formed for Euclidean:  96

Cophenet Correlation of DTW:  0.9381000700198061
Cophenet Correlation of Euclidean:  0.9644072500520264

*************Cluster_DTW***************

Adjusted Rand Index:  0.12664895510134777
Adjusted Mutual Info Score:  0.17262554368479796
Normalized Mutual Info Score:  0.41219861563148674
Homogeneity, Completeness, V-measure:  (0.6087323356804747, 0.31159717247775287, 0.4121986156314867)
Fowlkes Mallows Score:  0.2774075023817772


*************Cluster_Euclidean***************

Adjusted Rand Index:  0.08758828376404063
Adjusted Mutual Info Score:  0.17069724499139555
Normalized Mutual Info Score:  0.46563768684461465
Homogeneity, Completeness, V-measure:  (0.8309409368282037, 0.3234435076005567, 0.4656376868446147)
Fowlkes Mallows Score:  0.20658860604808352




## Average linkage clustering. The results are at the last 2 columns of the table below. Best results were obtained at inconsistency coefficient value 1.15

In [0]:
def get_linkage(org_dists, fun, method): 
  
  Z = fc.linkage(org_dists, metric = fun, method = method )         
  cophe_corr, _ = cophenet(Z, org_dists)

  return Z, cophe_corr

Z_dtw, cophe_dtw = get_linkage(dist_dtw, dtw, 'average')
Z_euc, cophe_euc = get_linkage(dist_euc, euclidean, 'average')

In [40]:
df['Cluster_DTW'] = fcluster(Z_dtw, 1.15)
df['Cluster_Euclidean'] = fcluster(Z_euc, 1.15)
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,B15,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,D9,D10,D11,D12,D13,D14,D15,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,E11,E12,E13,E14,E15,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,Target,Cluster_DTW,Cluster_Euclidean
0,-2,-2,-2,-2,-3,-2,-2,-2,-2,-2,-2,-1,-2,-3,-2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,81,79,79,80,79,81,80,79,79,79,...,0,0,1,0,0,1,0,-5,-4,-4,-4,-5,-5,-4,-4,-4,-4,-4,-4,-5,-4,-4,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,normal,8,30
1,6,42,-5,-4,2,0,-3,-6,-3,-2,-4,2,-1,-4,-4,-1,-3,4,-17,1,-5,-7,2,3,-9,2,-3,-5,5,1,79,80,74,93,74,78,83,71,76,86,...,-6,16,-5,7,6,-14,-3,4,53,-10,-6,-3,-6,-5,-11,-9,-8,-7,1,-6,-9,-12,-3,3,-1,-3,-4,0,-3,1,-4,-2,2,9,0,-4,5,normal,8,30
2,-2,0,-4,-2,-4,-2,1,-8,-1,-3,-3,-6,0,-5,-6,-6,2,-5,-2,4,4,-9,3,0,-6,8,-1,-9,5,-10,85,74,76,78,76,74,87,69,83,83,...,-2,6,-19,-1,13,-17,16,-5,1,-11,-6,-13,-9,-2,-20,-6,-14,-11,-20,-5,-16,-14,2,0,4,1,-10,-4,-10,-5,-8,-5,-3,6,2,7,-1,normal,8,30
3,-2,-5,0,-2,-6,2,-1,1,-3,-1,-5,-4,1,-9,-1,-10,4,1,-6,3,-9,-8,3,-8,-1,4,-14,-7,6,-9,86,73,79,87,73,84,82,78,88,81,...,5,-4,-17,17,3,-18,11,-7,-18,-10,-12,-21,-6,-8,-2,-13,-10,-21,-17,-7,-28,-15,-5,-2,-4,-1,-5,-9,0,-5,-3,-13,-15,-4,-2,-9,-11,normal,8,30
4,0,-2,2,1,1,-5,-1,-6,-5,-1,-3,-4,-2,3,-7,2,4,-9,-3,-1,-6,-6,-2,-10,5,5,-11,5,-3,-13,74,74,85,77,71,83,77,77,90,75,...,7,-23,-19,11,-18,-1,12,-9,-15,-4,-11,-2,-21,-17,-25,-21,-16,-19,-14,-11,-7,-30,-5,-3,-6,-10,-3,-5,-11,-10,-9,-6,-6,-4,-9,-1,-3,normal,8,30


## The evaluation results

In [41]:
print_results()

No. of clusters formed for DTW:  27
No. of clusters formed for Euclidean:  44

Cophenet Correlation of DTW:  0.9685274074327944
Cophenet Correlation of Euclidean:  0.989087963295629

*************Cluster_DTW***************

Adjusted Rand Index:  0.16323985141393638
Adjusted Mutual Info Score:  0.30427916148809175
Normalized Mutual Info Score:  0.41828231088584283
Homogeneity, Completeness, V-measure:  (0.5320223113191902, 0.3446090268375649, 0.4182823108858428)
Fowlkes Mallows Score:  0.32172282979106087


*************Cluster_Euclidean***************

Adjusted Rand Index:  0.18542223979639536
Adjusted Mutual Info Score:  0.29454295400719166
Normalized Mutual Info Score:  0.4496190457510156
Homogeneity, Completeness, V-measure:  (0.651781905122885, 0.3431762455883418, 0.44961904575101563)
Fowlkes Mallows Score:  0.31920237302381055




## Complete linkage clustering. The results are at the last 2 columns of the table below. Best results were obtained at inconsistency coefficient value 1.15

In [42]:
Z_dtw, cophe_dtw = get_linkage(dist_dtw, dtw, 'complete')
Z_euc, cophe_euc = get_linkage(dist_euc, euclidean, 'complete')

df['Cluster_DTW'] = fcluster(Z_dtw, 1.15)
df['Cluster_Euclidean'] = fcluster(Z_euc, 1.15)
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,B15,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,D9,D10,D11,D12,D13,D14,D15,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,E11,E12,E13,E14,E15,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,Target,Cluster_DTW,Cluster_Euclidean
0,-2,-2,-2,-2,-3,-2,-2,-2,-2,-2,-2,-1,-2,-3,-2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,81,79,79,80,79,81,80,79,79,79,...,0,0,1,0,0,1,0,-5,-4,-4,-4,-5,-5,-4,-4,-4,-4,-4,-4,-5,-4,-4,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,normal,12,16
1,6,42,-5,-4,2,0,-3,-6,-3,-2,-4,2,-1,-4,-4,-1,-3,4,-17,1,-5,-7,2,3,-9,2,-3,-5,5,1,79,80,74,93,74,78,83,71,76,86,...,-6,16,-5,7,6,-14,-3,4,53,-10,-6,-3,-6,-5,-11,-9,-8,-7,1,-6,-9,-12,-3,3,-1,-3,-4,0,-3,1,-4,-2,2,9,0,-4,5,normal,12,16
2,-2,0,-4,-2,-4,-2,1,-8,-1,-3,-3,-6,0,-5,-6,-6,2,-5,-2,4,4,-9,3,0,-6,8,-1,-9,5,-10,85,74,76,78,76,74,87,69,83,83,...,-2,6,-19,-1,13,-17,16,-5,1,-11,-6,-13,-9,-2,-20,-6,-14,-11,-20,-5,-16,-14,2,0,4,1,-10,-4,-10,-5,-8,-5,-3,6,2,7,-1,normal,12,16
3,-2,-5,0,-2,-6,2,-1,1,-3,-1,-5,-4,1,-9,-1,-10,4,1,-6,3,-9,-8,3,-8,-1,4,-14,-7,6,-9,86,73,79,87,73,84,82,78,88,81,...,5,-4,-17,17,3,-18,11,-7,-18,-10,-12,-21,-6,-8,-2,-13,-10,-21,-17,-7,-28,-15,-5,-2,-4,-1,-5,-9,0,-5,-3,-13,-15,-4,-2,-9,-11,normal,12,16
4,0,-2,2,1,1,-5,-1,-6,-5,-1,-3,-4,-2,3,-7,2,4,-9,-3,-1,-6,-6,-2,-10,5,5,-11,5,-3,-13,74,74,85,77,71,83,77,77,90,75,...,7,-23,-19,11,-18,-1,12,-9,-15,-4,-11,-2,-21,-17,-25,-21,-16,-19,-14,-11,-7,-30,-5,-3,-6,-10,-3,-5,-11,-10,-9,-6,-6,-4,-9,-1,-3,normal,12,16


## The evaluation results

In [43]:
print_results()

No. of clusters formed for DTW:  29
No. of clusters formed for Euclidean:  42

Cophenet Correlation of DTW:  0.844636486866201
Cophenet Correlation of Euclidean:  0.9155016364481732

*************Cluster_DTW***************

Adjusted Rand Index:  0.18995512048853444
Adjusted Mutual Info Score:  0.3449293241960455
Normalized Mutual Info Score:  0.45476873050034905
Homogeneity, Completeness, V-measure:  (0.6188715572365591, 0.35945420206369394, 0.454768730500349)
Fowlkes Mallows Score:  0.32870168272001166


*************Cluster_Euclidean***************

Adjusted Rand Index:  0.1636725355245904
Adjusted Mutual Info Score:  0.29727543518769534
Normalized Mutual Info Score:  0.4472671655713392
Homogeneity, Completeness, V-measure:  (0.6661115153306617, 0.3366606416988723, 0.4472671655713392)
Fowlkes Mallows Score:  0.2955001892093171


